In [1]:
#dependencies

import pandas as pd
import numpy as np
import requests
import json

moviedbapi = "87594855563381b8f9fa3b3224c9f19f"
idURL = "https://api.themoviedb.org/3/search/movie?query="
idURL1 = "&api_key="
detailURL = "https://api.themoviedb.org/3/movie/"
detailURL1 = "?api_key="
omdbapi = "e8913e4c"
omdb1 = "http://www.omdbapi.com/?t="
omdb2 = "&apikey="

In [2]:
#import csv file as pd dataframe

csvPath = 'movies.csv'
opening = pd.read_csv(csvPath)
# print(len(opening))
# opening.head()

In [3]:
#filters out movies that made at least $10000 in the first weekend

opening = opening.loc[(opening['opening']>999)]
# print(len(opening))
# opening.head()

In [4]:
#filters our rows with the budgets already populated and saves for later

budgetDF = opening.loc[(opening["budget"] > 0)]
# print(len(budgetDF))
# budgetDF.head()

In [5]:
# filter for movies that do not have budget data
nobudgetDF = opening.loc[(opening["budget"] == 0)]
# print(len(nobudgetDF))
# nobudgetDF.head()

In [6]:
#get ids from movieDB
#runs a query on key words to get ID that will be searched to grab a movie budget

idList = []
df = []

for index, row in nobudgetDF.iterrows():
#     print('--------------------------------------------------------------------------------')
    movie = row['title']
#     finalMovie = movie.replace(":","")
#     finalMovie = movie.replace(" ","%20")
    openGross = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    date = row['release']
    genre = row['genre']
    finalURL = idURL + movie + idURL1 + moviedbapi
#     print (finalURL)
    json_file = requests.get(finalURL).json()
    moviesJSON = json_file['results']
#     print (moviesJSON)
#     print (len(json_file['results']))

# grabs a list of all the release dates based on query
    try:
        dates = []
        for n in np.arange(len(moviesJSON)):
#             print (n)
            dt = moviesJSON[n]['release_date']
            dt = int(dt.replace("-",""))
#             print (dt)
            dates.append(dt)
#             print (f"success {n}")
#         print (dates)      
#         print (recent)
    except:
        pass
#         print ("no dates")

#tries to match the release date on the database and the csv

    try:
        for number in dates:
#             print (number)
            if date-30 <= number and number <= date + 30:
#                 print("DateMatch")
                ind = dates.index(number)
#                 print (ind)
#                 print (moviesJSON)
                recentMovie = moviesJSON[ind]
                idNumber = recentMovie['id']
#                 print (f"success {n}.{n}")
#                 movie = recentMovie['original_title']
#                 print(f'{movie} / {number} / {ind} / {idNumber} / {openGross}')
                movieDict = {"release":number,"title":movie, "gross":openGross, "id_number":idNumber, "domestic":domestic, 
                             "worldwide":worldwide, "genre":genre}
                df.append(movieDict)
            else:
                continue
    
    except:
        print ("no id")

idDF = pd.DataFrame(df)
# print (len(idDF))
# pd.options.display.max_rows = 999
# idDF.head()

In [7]:
#get budgets for the movies without the budget that have budgets on the database
budgets = []
for index, row in idDF.iterrows():
    genre = row['genre']
    gross = row['gross']
    title = row['title']
    domestic = row['domestic']
    worldwide = row['worldwide']

    try:
        
        detailsURL = detailURL + str(row['id_number']) + detailURL1 + moviedbapi
#     print (detailsURL)
        details = requests.get(detailsURL).json()
        budget = details['budget']
        release = details['release_date']
#         print (f"Budget Found = {budget}")
        fullDict = {"title": title, "opening":gross, "budget":budget, "domestic":domestic, 
                    "genre":genre, "release":release, "worldwide":worldwide}
        budgets.append(fullDict)
    except:
        pass
#     print (genre)
    
addbudgetDF = pd.DataFrame(budgets)
addbudgetDF = addbudgetDF.loc[(addbudgetDF['budget']>0)]
# print(len(addbudgetDF))
# addbudgetDF.head()

In [8]:
# add budget to original dataframe and combine with new budgets found in MovieDB
budgetFinal = []

for index, row in budgetDF.iterrows():
    release = row['release']
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    moviesData = {"release": release, "title":title, "genre":genre, "budget":budget, 
                  "opening":opening, "domestic":domestic, "worldwide":worldwide}
    budgetFinal.append(moviesData)
    
for index, row in addbudgetDF.iterrows():
    release = row['release']
    release = release.replace('-','')
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    moviesData = {"release": release, "title":title, "genre":genre, "budget":budget, 
                  "opening":opening, "domestic":domestic, "worldwide":worldwide}
    budgetFinal.append(moviesData)
    
moviesDF = pd.DataFrame(budgetFinal)
# print (len(moviesDF))
# moviesDF.head()

In [9]:
# get ratings (out of 100) and rated (G, PG, PG-13, R)
moviesFinal =[]
for index, row in moviesDF.iterrows():
    release = row['release']
    name = row['title']
    #turns title name into hashtags and removes punctuation
    title = name.replace(" ","")
    title = title.replace(":","")
    title = title.replace("’","")
    title = title.replace(".","")
    title = title.replace(",","")
    title = title.replace("!","")
    title = title.replace("?","")
    title = title.replace("-","")
    title = title.replace("&","")
    title = title.replace("$","")
    title = "#" + title
#     print (title)

    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    rbratio = opening/budget
    omdbURL = omdb1 + name + omdb2 + omdbapi
#     print (omdbURL)
    ratingJSON = requests.get(omdbURL).json()
    ratings = []
    finalRatings = []
    try:
        # attempt to get ratings from IMDB, Rotten Tomatoes, Metacritic
        
        rated = ratingJSON['Rated']
        ratingJSON = ratingJSON['Ratings']
#         print (ratingJSON)
        for x in np.arange(len(ratingJSON)):
            value = ratingJSON[x]['Value']
            ratings.append(value)
#         print (ratings)
        for number in ratings:
#             print (number)
#             print (number.replace("'",""))
            numbers = number.replace("'","")
#             print (numbers)
            numbers = numbers.replace("/100","")
            numbers = numbers.replace("/10","")
            numbers = numbers.replace(".","")
            numbers = numbers.replace("%","")
            finalRatings.append(int(numbers))
    except:
        finalRatings = 0
    try:
        avgRating = np.mean(finalRatings).round(2)
#         print (avgRating)
    except:
        avgRating = 0
# create final dataframe    
    moviesFinalDict = {"release": release, "title":title, "genre":genre, "budget":budget, 
                      "opening":opening, "ratio":rbratio, "domestic":domestic, 
                       "worldwide":worldwide, "value": avgRating, "rated":rated}
    moviesFinal.append(moviesFinalDict)
moviesFinalDF = pd.DataFrame(moviesFinal)

moviesFinalDF = moviesFinalDF[['title','genre','release','rated','value','budget','opening','domestic','worldwide', 'ratio']]
moviesFinalDF = moviesFinalDF.dropna(how='any')

# print (len(moviesFinalDF))
# moviesFinalDF

C:\Users\mcyee\Anaconda3\envs\pythondata\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\mcyee\Anaconda3\envs\pythondata\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
# add scores for movie and make a customized score

moviesFinalDF['zrating'] = ''
moviesFinalDF['zbudget'] = ''
moviesFinalDF['zopening'] = ''
moviesFinalDF['ztotal'] = ''
ratingsMean = np.mean(moviesFinalDF['value'])
ratingsSD = np.std(moviesFinalDF['value'])
budgetMean = np.mean(moviesFinalDF['budget'])
budgetSD = np.std(moviesFinalDF['budget'])
openingMean = np.mean(moviesFinalDF['opening'])
openingSD = np.std(moviesFinalDF['opening'])

for index, movie in moviesFinalDF.iterrows():
    zrat = (movie['value'] - ratingsMean)/ratingsSD
    moviesFinalDF.set_value(index, 'zrating', zrat)
    zbud = (movie['budget'] - budgetMean)/budgetSD
    moviesFinalDF.set_value(index, 'zbudget', zbud)
    zope = (movie['opening'] - openingMean)/openingSD
    moviesFinalDF.set_value(index, 'zopening', zope)
    ztot = zrat + zbud + (2 * zope)
    moviesFinalDF.set_value(index, 'ztotal', ztot)

moviesFinalDF = moviesFinalDF.sort_values(by =['value'])
moviesFinalDF = moviesFinalDF.loc[(moviesFinalDF["budget"] >= 10000000)]
# print (len(moviesFinalDF))
moviesFinalDF

,title,genre,release,rated,value,budget,opening,domestic,worldwide,ratio,zrating,zbudget,zopening,ztotal
40,#Jìyìdàshi,Thriller/Suspense,20170428,R,0.00,20000000,224942,587470,43326383,0.011247,-2.89829,-0.523493,-0.581893,-4.58557
87,#TylerPerrysBoo2AMadeaHalloween,Comedy,20171020,PG-13,0.00,20000000,21226953,47319572,47879572,1.061348,-2.89829,-0.523493,0.0336204,-3.35454
135,#Gongfuyujia,Action,20170127,PG,0.00,65000000,111979,362657,254569616,0.001723,-2.89829,0.237558,-0.585204,-3.83114
64,#TheEmojiMovie,Adventure,20170728,PG,17.00,50000000,24531923,86089513,216807482,0.490638,-2.05551,-0.0161257,0.13048,-1.81067
111,#JustGettingStarted,Comedy,20171208,PG-13,22.00,22000000,3201459,6069605,6709821,0.145521,-1.80763,-0.489668,-0.49466,-3.28662
14,#Rings,Horror,20170203,PG-13,25.67,25000000,13002632,27793018,82991460,0.520105,-1.62569,-0.438931,-0.207413,-2.47945
93,#TheSnowman,Thriller/Suspense,20171020,R,27.00,35000000,3372565,6670765,43232849,0.096359,-1.55976,-0.269809,-0.489645,-2.80885
89,#Geostorm,Action,20171020,PG-13,29.33,100000000,13707376,33700160,220800160,0.137074,-1.44424,0.829486,-0.186759,-0.988276
120,#FiftyShadesFreed,Drama,20180209,R,29.33,55000000,38560195,98382615,359382615,0.701094,-1.44424,0.0684355,0.541612,-0.292584
10,#FiftyShadesDarker,Drama,20170210,R,29.67,55000000,46607250,114434010,381081967,0.847405,-1.42739,0.0684355,0.77745,0.195948


In [11]:
# export CSV
moviesFinalDF.to_csv('moviesFinal.csv', encoding='utf-8', index=False)